In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Subset, DataLoader
import torchvision
from torch import optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models

import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
eng_characters = [chr(i) for i in range(ord('a'), ord('z')+1)]  # English lowercase letters
eng_characters += [chr(i) for i in range(ord('A'), ord('Z')+1)]  # English uppercase letters
eng_characters += [chr(i) for i in range(ord('0'), ord('9')+1)]  # Digits
eng_characters += ['?','<','>']
# Add other characters as needed, such as punctuation marks, special symbols, etc.

print(eng_characters)
# Define the range of Unicode code points for Telugu characters
TELUGU_START = 0x0C00
TELUGU_END = 0x0C7F

# Generate all Telugu characters
telugu_characters = [chr(code_point) for code_point in range(TELUGU_START, TELUGU_END + 1)]
telugu_characters += ['?','<','>']

print(telugu_characters)



['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', '<', '>']
['ఀ', 'ఁ', 'ం', 'ః', 'ఄ', 'అ', 'ఆ', 'ఇ', 'ఈ', 'ఉ', 'ఊ', 'ఋ', 'ఌ', '\u0c0d', 'ఎ', 'ఏ', 'ఐ', '\u0c11', 'ఒ', 'ఓ', 'ఔ', 'క', 'ఖ', 'గ', 'ఘ', 'ఙ', 'చ', 'ఛ', 'జ', 'ఝ', 'ఞ', 'ట', 'ఠ', 'డ', 'ఢ', 'ణ', 'త', 'థ', 'ద', 'ధ', 'న', '\u0c29', 'ప', 'ఫ', 'బ', 'భ', 'మ', 'య', 'ర', 'ఱ', 'ల', 'ళ', 'ఴ', 'వ', 'శ', 'ష', 'స', 'హ', '\u0c3a', '\u0c3b', '\u0c3c', 'ఽ', 'ా', 'ి', 'ీ', 'ు', 'ూ', 'ృ', 'ౄ', '\u0c45', 'ె', 'ే', 'ై', '\u0c49', 'ొ', 'ో', 'ౌ', '్', '\u0c4e', '\u0c4f', '\u0c50', '\u0c51', '\u0c52', '\u0c53', '\u0c54', 'ౕ', 'ౖ', '\u0c57', 'ౘ', 'ౙ', 'ౚ', '\u0c5b', '\u0c5c', '\u0c5d', '\u0c5e', '\u0c5f', 'ౠ', 'ౡ', 'ౢ', 'ౣ', '\u0c64', '\u0c65', '౦', '౧', '౨', '౩', '౪', '౫', '౬', '౭', '౮', '౯'

In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

train_file = '/kaggle/input/aksharantar2/aksharantar_sampled/tel/tel_train.csv'
valid_file = '/kaggle/input/aksharantar2/aksharantar_sampled/tel/tel_valid.csv'
# Load train and validation datasets

# Load the datasets
train_dataset = pd.read_csv(train_file, header=None)
valid_dataset = pd.read_csv(valid_file, header=None)

# Extract the source and target columns
source_data = train_dataset.iloc[:, 0].values
target_data = train_dataset.iloc[:, 1].values


In [4]:
source_data = ['<' + s + '>' for s in source_data]
target_data = ['<' + t + '>' for t in target_data]

In [5]:
loc = 1
e,t = train_dataset.iloc[loc]
char_to_index_eng = {char: i for i, char in enumerate(eng_characters)}
index_to_char_eng = {i: char for char, i in char_to_index_eng.items()}
def char_to_tensor_eng(word, char_to_index_eng):
    return torch.tensor([char_to_index_eng[char] for char in word], dtype=torch.long)

print(e)
print(char_to_tensor_eng(e,char_to_index_eng))

char_to_index_tel = {char: i for i, char in enumerate(telugu_characters)}
index_to_char_tel = {i: char for char, i in char_to_index_tel.items()}
def char_to_tensor_tel(word, char_to_index_tel):
    return torch.tensor([char_to_index_tel[char] for char in word], dtype=torch.long)

print(t)
t1 = char_to_tensor_tel(t,char_to_index_tel)
print(t1)

characters = [telugu_characters[char_int.item()] for char_int in t1]
string = ''.join(characters)
# Print the characters
print("Characters:", string)
# for batch_idx, (data, target) in enumerate(train_loader):
#     print(f"Batch {batch_idx}:")
#     print("Data shape:", data.shape)
#     print("Target shape:", target.shape)
#     # You can print more details about the data and target if needed
#     break  # This will break after printing the first batch

vastadira
tensor([21,  0, 18, 19,  0,  3,  8, 17,  0])
వస్తాదిరా
tensor([53, 56, 77, 36, 62, 38, 63, 48, 62])
Characters: వస్తాదిరా


In [6]:

source = [char_to_tensor_eng(s, char_to_index_eng) for s in source_data]
target = [char_to_tensor_tel(s, char_to_index_tel) for s in target_data]

In [7]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset
# Pad the tensors to the same length

source_padded = pad_sequence(source, batch_first=True, padding_value=62)
target_padded = pad_sequence(target, batch_first=True, padding_value=128)

# Find the maximum length
max_length = max(source_padded.size(1), target_padded.size(1))

# Pad source and target to the same length
source_padded = torch.nn.functional.pad(source_padded, (0, max_length - source_padded.size(1)), value=62)
target_padded = torch.nn.functional.pad(target_padded, (0, max_length - target_padded.size(1)), value=128)

# Add 62 at the beginning and 63 at the end of each item in source_padded
# source_padded = torch.cat((torch.full((source_padded.shape[0], 1), 63, dtype=torch.long), 
#                            source_padded, torch.full((source_padded.shape[0], 1), 64, dtype=torch.long)), dim=1)
# target_padded = torch.cat((torch.full((target_padded.shape[0], 1), 129, dtype=torch.long), 
#                            target_padded, torch.full((target_padded.shape[0], 1), 130, dtype=torch.long)), dim=1)

In [8]:
target_padded[0]

tensor([129,  53,  48,  77,  23,  62,  50,  53,  62,  48,  63,  40,  71, 130,
        128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,
        128, 128])

In [9]:
def t2p(s,t):
    characters = [eng_characters[char_int.item()] for char_int in s]
    string1 = ''.join(characters)
#     print(string)
    characters = [telugu_characters[char_int.item()] for char_int in t]
    string2 = ''.join(characters)
    print(f"{string1} {len(string1)} : {string2} {len(string2)} :")
t2p(source_padded[0],target_padded[0])

<vargaalavaarine>????????????? 30 : <వర్గాలవారినే>???????????????? 30 :


In [ ]:
# Create a TensorDataset
dataset = TensorDataset(source_padded, target_padded)

# Create a DataLoader
batch_size = 64
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Example usage of the DataLoader
cnt=1
for batchid,(s,t) in enumerate(tqdm(train_dataloader)):
    #source_batch, target_batch = batch
    # Your training code here
    print(s[0])
    for item_s, item_t in zip(s, t):
#     print(f"{batchid}:")
        t2p(item_s,item_t)
#     print(s)
#     cnt+=1
#     if(cnt>2):break
    if(batchid>0): break


In [19]:
#valid_dataset = pd.read_csv(valid_file, header=None)

# Extract the source and target columns
val_source_data = valid_dataset.iloc[:, 0].values
val_target_data = valid_dataset.iloc[:, 1].values

val_source_data = ['<' + s + '>' for s in val_source_data]
val_target_data = ['<' + t + '>' for t in val_target_data]

val_source = [char_to_tensor_eng(s, char_to_index_eng) for s in val_source_data]
val_target = [char_to_tensor_tel(s, char_to_index_tel) for s in val_target_data]

val_source_padded = pad_sequence(val_source, batch_first=True, padding_value=62)
val_target_padded = pad_sequence(val_target, batch_first=True, padding_value=128)

# Find the maximum length
max_length = max(val_source_padded.size(1), val_target_padded.size(1))

# Pad source and target to the same length
val_source_padded = torch.nn.functional.pad(val_source_padded, (0, max_length - val_source_padded.size(1)), value=62)
val_target_padded = torch.nn.functional.pad(val_target_padded, (0, max_length - val_target_padded.size(1)), value=128)
# Create a TensorDataset
val_dataset = TensorDataset(val_source_padded, val_target_padded)


valid_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [12]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, cell_type, batch_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        
        # Character embedding layer
        self.embedding = nn.Embedding(input_size, embedding_size)
        
        # Encoder RNN
        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, batch_first=True)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, batch_first=True)
    
    def forward(self, x, hidden):
        embedded = self.embedding(x)
        # embedded shape: (seq_length:max_length, N(batch size:32), embedding_size:100)
        output, hidden = self.rnn(embedded, hidden)
        return hidden
    
    def getInitialState(self):
        return torch.zeros(num_layers,batch_size,hidden_size).to(device)


class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, 
                 output_size, num_layers, cell_type):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Devanagari character embedding layer
        self.embedding = nn.Embedding(output_size, embedding_size)
        
        # Decoder RNN
        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, batch_first=True)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)
    
    def forward(self, x, hidden):
        # shape of x: (N) but required (1, N)
#         x = x.unsqueeze(0)
        
        embedded = self.embedding(x)
        # embedded shape: (1, N, embedding_size) #n * 1 * embd
        output, hidden = self.rnn(embedded, hidden)  
        # shape of outputs: (N*1*131)
        predictions = self.softmax(self.fc(output))
        # shape of predictionos:  (1, N, length of tel_char)
#         predictions = predictions.squeeze(0)

        
        return predictions, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, cell_type):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
    
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.shape[0]
        target_len = target.shape[1]
        target_letter_size = 131
#         print(source.shape)
#         print(target.shape)
        
        outputs = torch.zeros(target_len, batch_size, target_letter_size).to(device)
        
        # Encoder forward pass
        if(cell_type == "LSTM"):
            hidden = (self.encoder.getInitialState(), self.encoder.getInitialState())
        else:
            hidden = self.encoder.getInitialState()
#         hidden.to(device)
        hidden = self.encoder(source, hidden)
        
#         print(hidden.shape)
        x = target[:,0].view(batch_size, 1)
#         print( "target :",x.shape)
#         print(x)
        
#         # Decoder initial hidden state
#         decoder_hidden = encoder_hidden        
#         # Decoder input (start token)
#         decoder_input = torch.tensor([[SOS_token]] * batch_size)      
        # Decoder forward pass
    
        for t in range(0, target_len):
            output, hidden = self.decoder(x, hidden)
            output = output[:, -1, :]
            outputs[t] = output
            best_guess = output.argmax(1)
            best_guess = best_guess.view(batch_size, 1) 
            x = target[:,t].view(batch_size,1) if np.random.rand() < teacher_forcing_ratio else best_guess
        
        return outputs


In [13]:
input_size = len(char_to_index_eng)
print(input_size)
char_to_index_eng['?']

65


62

In [14]:
# Define the model architecture
from torch.utils.tensorboard import SummaryWriter
input_size = len(char_to_index_eng)
hidden_size = 256
embedding_size = 256
output_size = len(char_to_index_tel)
num_layers = 2
cell_type = 'LSTM'
LEARNING_RATE = 0.0001
NUM_EPOCHS = 20

encoder_net = Encoder(input_size, embedding_size, hidden_size, num_layers, cell_type, batch_size).to(device)
decoder_net = Decoder(output_size, embedding_size, hidden_size, 
                 output_size, num_layers, cell_type).to(device)
model = Seq2Seq(encoder_net, decoder_net, cell_type).to(device)

# Define the loss function and optimizer
criterion = nn.NLLLoss()
# criterion = nn.CrossEntropyLoss(ignore_index = char_to_index_tel['?'])
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

writer = SummaryWriter(f'runs/Loss_plot')
step = 0
# Train the model
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    for batchid,(inp_data,target) in enumerate(tqdm(train_dataloader)):
#         print("batch: ", batchid)
        inp_data = inp_data.to(device)
        target = target.to(device)
#         print("inp_data")
#         print(inp_data.shape)
        output = model(inp_data, target)
        indices = output.argmax(2)
        #output shape: (trg_len, batch_size, output_dim)
#         print("output_data")
#         print(output.shape, output.shape[0],output.shape[1], output.shape[2])
#         print(indices.shape)
        if(batchid%400==0):
            t2p(inp_data[0],target[0])
            t2p(inp_data[0],indices[:,0])
#             print(indices[0])
#             print(inp_data.shape, target.shape,output.shape, indices.shape)
#         output = output[1:].reshape(-1, output.shape[2])
#         target = target[1:].reshape(-1)        
#         output = output[:, -1, :]
#         print("output:  ", output.shape)
#         print("target:  ", target.shape)
#         target = target[1:]



        # Flatten the output and target tensors to compute the loss
        optimizer.zero_grad()
        loss = 0
        for i in range(output.shape[0]):
            loss += criterion(output[i], target[:,i])
        
        total_loss += loss.item()/ len(inp_data[0])
        # Backpropagation
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=1)
        optimizer.step()
        step+=1
        writer.add_scalar('Training loss', loss, global_step=step)


    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataloader)}")

# # Evaluate the model on the validation dataset
# model.eval()
# total_val_loss = 0
# with torch.no_grad():
#     for input_tensor, target_tensor in valid_loader:
#         output = model(input_tensor, target_tensor)
#         output_dim = output.shape[-1]

#         # Flatten the output and target tensors to compute the loss
#         val_loss = criterion(output.view(-1, output_dim), target_tensor.view(-1))
#         total_val_loss += val_loss.item()

#     print(f"Validation Loss: {total_val_loss / len(valid_loader)}")


2024-05-15 17:55:24.860987: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 17:55:24.861083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 17:55:25.035501: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
  0%|          | 0/800 [00:00<?, ?it/s]

<shivagamila>????????????????? 30 : <శివగామిలా>??????????????????? 30 :
<shivagamila>????????????????? 30 : ఏఏఏౄఏఏ఻ఏఏఏ౵>>౤౤౤౤౤ౄ౤౤౤౤ౢ౤౤౤౤౤౤ 30 :


 50%|█████     | 404/800 [00:20<00:17, 22.14it/s]

<jeevaale>???????????????????? 30 : <జీవాలే>?????????????????????? 30 :
<jeevaale>???????????????????? 30 : <క్్్్>>?????????????????????? 30 :


100%|██████████| 800/800 [00:38<00:00, 20.83it/s]


...

<gudamargam>?????????????????? 30 : <గుదమార్గం>??????????????????? 30 :
<gudamargam>?????????????????? 30 : <గుదమరర్గం>??????????????????? 30 :


 50%|█████     | 402/800 [00:18<00:18, 21.62it/s]

<kasukunna>??????????????????? 30 : <కాసుకున్నా>?????????????????? 30 :
<kasukunna>??????????????????? 30 : <కసుుకున్నా>?????????????????? 30 :


100%|██████████| 800/800 [00:36<00:00, 21.76it/s]

Epoch 20, Loss: 4.939445465505123


In [36]:
def calculate_validation_loss(model, criterion, valid_dataloader):
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    accuracy = 0
    step = 0
    with torch.no_grad():
        for batchid,(inp_data,target) in enumerate(tqdm(valid_dataloader)):
            inp_data = inp_data.to(device)
            target = target.to(device)

            output = model(inp_data, target)
            indices = output.argmax(2)
            output_dim = output.shape[-1]
            if(batchid%2==0):
                t2p(inp_data[0],target[0])
                t2p(inp_data[0],indices[:,0])
                print((indices[:,0] == target[0]).sum().item())

            # Flatten the output and target tensors to compute the loss
            val_loss = 0
            total_correct = 0
            for i in range(output.shape[0]):
                val_loss += criterion(output[i], target[:, i])
                total_correct += ((indices[:, i] == target[i]).sum().item() == len(target[i]))
            step +=1
            total_val_loss += val_loss.item() / len(inp_data[0])
            accuracy += total_correct
            writer.add_scalar('validation loss', val_loss, global_step=step)

    avg_val_loss = total_val_loss / len(valid_dataloader)
    accuracy = accuracy #/ len(valid_dataloader)
    return avg_val_loss, accuracy
ll, aa = calculate_validation_loss(model, criterion, valid_dataloader)
print("validation lass: ", ll)
print("accuracy: ", aa, aa/len(valid_dataloader))

  8%|▊         | 5/64 [00:00<00:01, 42.33it/s]

<bheeshmudini>???????????????? 30 : <భీష్ముడిని>?????????????????? 30 :
<bheeshmudini>???????????????? 30 : <భీష్ముడిని>?????????????????? 30 :
30
<thiraalu>???????????????????? 30 : <తీరాలు>?????????????????????? 30 :
<thiraalu>???????????????????? 30 : <తిరాలు>?????????????????????? 30 :
29
<evans>??????????????????????? 30 : <ఇవాన్స్>????????????????????? 30 :
<evans>??????????????????????? 30 : <ఎవన్స్్>????????????????????? 30 :
25
<shaastravettalalo>??????????? 30 : <శాస్త్రవేత్తలలో>????????????? 30 :
<shaastravettalalo>??????????? 30 : <శాస్త్రవెత్తలలో>????????????? 30 :
29
<empikacheyadamtho>??????????? 30 : <ఎంపికచేయడంతో>???????????????? 30 :
<empikacheyadamtho>??????????? 30 : <ఎంపికచేయడంతో>???????????????? 30 :
30


 31%|███▏      | 20/64 [00:00<00:00, 45.39it/s]

<pattanaabhivruddikosam>?????? 30 : <పట్టణాభివృద్దికోసం>?????????? 30 :
<pattanaabhivruddikosam>?????? 30 : <పట్టనాభివృద్ద్యోంద్?????????? 30 :
24
<baapuu>?????????????????????? 30 : <బాపూ>???????????????????????? 30 :
<baapuu>?????????????????????? 30 : <బాపూ>???????????????????????? 30 :
30
<katholik>???????????????????? 30 : <కాథలిక్>????????????????????? 30 :
<katholik>???????????????????? 30 : <కథోలిక్>????????????????????? 30 :
28
<haetuvu>????????????????????? 30 : <హేతువు>?????????????????????? 30 :
<haetuvu>????????????????????? 30 : <హేతువు>?????????????????????? 30 :
30
<juntipalli>?????????????????? 30 : <జుంటిపల్లి>?????????????????? 30 :
<juntipalli>?????????????????? 30 : <జుంటిపా్లి>?????????????????? 30 :
29


 39%|███▉      | 25/64 [00:00<00:00, 45.44it/s]

...

<rayagala>???????????????????? 30 : <రాయగల>??????????????????????? 30 :
<rayagala>???????????????????? 30 : <రాయగల>??????????????????????? 30 :
30
<tharkam>????????????????????? 30 : <తర్కం>??????????????????????? 30 :
<tharkam>????????????????????? 30 : <తర్కం>??????????????????????? 30 :
30
<sagamgaa>???????????????????? 30 : <సగంగా>??????????????????????? 30 :
<sagamgaa>???????????????????? 30 : <సంగగా>??????????????????????? 30 :
28
<pyramidla>??????????????????? 30 : <పిరమిడ్ల>???????????????????? 30 :
<pyramidla>??????????????????? 30 : <ప్యమిద్ల>???????????????????? 30 :
27
<vaethanamtho>???????????????? 30 : <వేతనంతో>????????????????????? 30 :
<vaethanamtho>???????????????? 30 : <వేతనంతో>????????????????????? 30 :
30


 94%|█████████▍| 60/64 [00:01<00:00, 46.12it/s]

<churna>?????????????????????? 30 : <చూర్ణ>??????????????????????? 30 :
<churna>?????????????????????? 30 : <చుర్న>??????????????????????? 30 :
28
<samakurchenduku>????????????? 30 : <సమకూర్చేందుకు>??????????????? 30 :
<samakurchenduku>????????????? 30 : <సమకుర్చేందుకు>??????????????? 30 :
29
<prathamikamaina>????????????? 30 : <ప్రాథమికమైన>????????????????? 30 :
<prathamikamaina>????????????? 30 : <ప్రతమమిమైైన>????????????????? 30 :
26
<dheshaalaloo>???????????????? 30 : <దేశాలలో>????????????????????? 30 :
<dheshaalaloo>???????????????? 30 : <ధేశాలలో>????????????????????? 30 :
29
<fuds>???????????????????????? 30 : <ఫుడ్స్>?????????????????????? 30 :
<fuds>???????????????????????? 30 : <ఫడడ్స్>?????????????????????? 30 :
29


100%|██████████| 64/64 [00:01<00:00, 45.30it/s]

<varudu>?????????????????????? 30 : <వరుడు>??????????????????????? 30 :
<varudu>?????????????????????? 30 : <వరుడు>??????????????????????? 30 :
30
<zamaath>????????????????????? 30 : <జమాత్>??????????????????????? 30 :
<zamaath>????????????????????? 30 : <జాాత్>??????????????????????? 30 :
29
validation lass:  0.17087744337817037
accuracy:  776 12.125


In [41]:
print(len(valid_dataloader))

64
